In [2]:
# Set notebook to use only one GPU
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


# Model

After pre-processing is complete and the data has been stored in a file, load data and train the model.

The format of the data is a np array of shape (total_runs_from_all_subjects, n_channels, n_timestamps)

Which translates to (5184, 22, 1125) for this dataset and these params.

In [34]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from livelossplot import PlotLossesKeras
from models_bachelors import *
from file_functions import *
import tensorflow as tf
import keras_tuner as kt
dataset = load('all_subject_runs_no_preprocess')
lockbox = load('lockbox')['data']
loaded_inputs = dataset['inputs']
loaded_targets = dataset['targets']

def remove_lockbox(loaded_inputs, loaded_targets, lockbox):
    inputs = loaded_inputs.copy()
    targets = loaded_targets.copy()
    per_sbj_lockbox_inputs = []
    per_sbj_lockbox_targets = []
    for i in range(lockbox.shape[0]):
        subj_inputs = inputs[i, lockbox[i,:], :, :]   
        subj_targets = targets[i, lockbox[i,:], :]
        per_sbj_lockbox_inputs.append(subj_inputs)
        per_sbj_lockbox_targets.append(subj_targets)

    per_subj_keep_inputs = []
    per_subj_keep_targets = []
    for i in range(lockbox.shape[0]):
        subj_keep_inputs = np.delete(inputs[i, :, :, :], obj=lockbox[i, :], axis=0)
        subj_keep_targets = np.delete(targets[i, :, :], obj=lockbox[i, :], axis=0)
        per_subj_keep_inputs.append(subj_keep)
        per_subj_keep_targets.append(subj_keep_targets)

    return np.vstack(per_subj_keep_inputs), np.vstack(per_subj_keep_targets)
    

# Hyperparameter Tuning

In [ ]:
n_epochs= 100
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
callbacks = [early_stopping]
# There's an error in MCDropConnect so it's out of the list for now
methods = ['mcdropout', 'mcdropconnect']
subject_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8]

# Training loop for MCDropout and MCDropconnect models
for method in methods:
    # This loop leaves one subject for testing (denoted by the number in the name of the weights file).
    # Then it combines all the subject trials such that shape is now (8 * 576, 22, 1125).
    # Then selects 10% of this as the validation set.
    subject_id = 0
    train_ids = subject_ids[:]
    train_ids.remove(subject_id)
    train_inputs = np.vstack(loaded_inputs[train_ids])
    train_targets = np.vstack(loaded_targets[train_ids])        
    X_train, X_val, Y_train, Y_val = train_test_split(train_inputs, train_targets, test_size=0.1)
    # Do hyperparam tuning (SUBJECT 0)
    best_model = tune_model(X_train, X_val, Y_train, Y_val, method, callbacks)

In [4]:
# Finding tuning results

mcdropout_tuner = kt.GridSearch(build_dropout_model,
                    objective='val_loss',
                    max_trials=100,
                    directory=f'mcdropout/tuning',
                    project_name='untitled_project')

mcdropconnect_tuner = kt.GridSearch(build_dropconnect_model,
                    objective='val_loss',
                    max_trials=100,
                    directory=f'mcdropconnect/tuning',
                    project_name='untitled_project')

mcdropout_tuner.reload()
mcdropconnect_tuner.reload()
dropout_best_hps = mcdropout_tuner.get_best_hyperparameters(num_trials=1)[0]
dropconnect_best_hps = mcdropconnect_tuner.get_best_hyperparameters(num_trials=1)[0]


# 0.2 is optimal for both

Reloading Tuner from mcdropout/tuning/untitled_project/tuner0.json
Reloading Tuner from mcdropconnect/tuning/untitled_project/tuner0.json


# Training

In [35]:
n_epochs= 100
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
# There's an error in MCDropConnect so it's out of the list for now
methods = ['mcdropout', 'mcdropconnect']
subject_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8]


# Training loop for MCDropout and MCDropconnect methods
for method in methods:
    directory = f'{method}/weights'
    # This loop leaves one subject for testing (denoted by the number in the name of the weights file).
    # Then it combines all the subject trials such that shape is now (8 * 576, 22, 1125).
    # Then selects 10% of this as the validation set. Then it trains diff. model on each set of train subjects.
    for subject_id in subject_ids:
        checkpoint_path = f'{directory}/test_subject_{subject_id}.ckpt'
        saving_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)
        train_ids = subject_ids[:]
        train_ids.remove(subject_id)
        test_subj_lockbox = lockbox[subject_id]
        inputs, targets = remove_lockbox(loaded_inputs, loaded_targets, test_subj_lockbox)
        X_train, X_val, Y_train, Y_val = train_test_split(inputs, targets,test_size=0.1)
        
        model = mcdropout_tuner.hypermodel.build(dropout_best_hps) if method == 'mcdropout' else mcdropconnect_tuner.hypermodel.build(dropconnect_best_hps)
        history = model.fit(X_train, Y_train, epochs=n_epochs, validation_data=[X_val, Y_val],
                           callbacks=[early_stopping, saving_callback, PlotLossesKeras()])
                

(9, 576, 22, 1125)


(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)
(9, 576, 22, 1125)
(4152, 22, 1125)


In [ ]:
print('Done')